In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic_2d
from gala.mpl_style import hesperia_r

from totoro.data import apogee, galah
from totoro.config import galcen_frame

In [ ]:
mh_am_bins = (np.arange(-2., 0.75, 0.01),
              np.arange(-0.18, 0.4, 0.005))

In [ ]:
zvz_bins = (np.arange(-100, 100+1e-3, 1.5),
            np.arange(-2, 2+1e-3, 0.05))

# APOGEE

In [ ]:
apogee_rgb = apogee.filter({'LOGG': (1, 3.4),
                            'TEFF': (3500, 6500)})

apogee_ms = apogee.filter({'LOGG': (3.75, 5),
                           'TEFF': (3500, 6500)})

In [ ]:
apogee_mh_alpham_nodes = np.array([
    [0.6, -0.05],
    [0.6, 0.04],
    [0.15, 0.04],
    [-0.5, 0.13],
    [-0.9, 0.13],
    [-1., 0.07],
    [-0.2, -0.1],
    [0.2, -0.1],
    [0.6, -0.05]]
)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for i, d in enumerate([apogee, apogee_ms, apogee_rgb]):
    ax = axes[i]

    H, xe, ye = np.histogram2d(d.t['TEFF'], 
                               d.t['LOGG'],
                               bins=(np.arange(3000, 7000+1e-3, 50),
                                     np.arange(0, 5.5+1e-3, 0.02)))

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)
    
ax.set_xlim(7000, 3000)
ax.set_ylim(5.5, 0)

#     ax.set_xlabel('metallicity, [M/H]')
#     ax.set_ylabel(r'alpha abundance, [$\alpha$/M]')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
for d in [apogee_ms, apogee_rgb]:
    fig, ax = plt.subplots(figsize=(8, 5))

    H, xe, ye = np.histogram2d(d.t['M_H'], 
                               d.t['ALPHA_M'],
                               bins=mh_am_bins)

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)

    ax.plot(apogee_mh_alpham_nodes[:, 0], 
            apogee_mh_alpham_nodes[:, 1],
            marker='', color='#777777', ls='--', lw=1.5)
    ax.set_xlim(-1.5, 0.7)
    ax.set_ylim(-0.12, 0.32)
    ax.set_xlabel('metallicity, [M/H]')
    ax.set_ylabel(r'alpha abundance, [$\alpha$/M]')

    fig.tight_layout()
    fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5.5), 
                         sharex=True, sharey=True)

for i, d in enumerate([apogee, apogee_ms, apogee_rgb]):
    ax = axes[i]
    
    galcen = d.c.transform_to(galcen_frame)
    H, xe, ye = np.histogram2d(galcen.v_z.to_value(u.km/u.s), 
                               galcen.z.to_value(u.kpc),
                               bins=zvz_bins)

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)
    
ax.set_xlim(-100, 100)
ax.set_ylim(-2, 2)

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

axes[0].set_title('full sample')
axes[1].set_title('main sequence')
axes[2].set_title('red giants')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15.5, 5.5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

for i, d in enumerate([apogee, apogee_ms, apogee_rgb]):
    ax = axes[i]
    
    e = d.get_elem_ratio('MG_FE')
    eivar = 1 / d.t['MG_FE_ERR']**2
    mask = (e > -3) & (e < 3)
    
    galcen = d.c.transform_to(galcen_frame)
    stat1 = binned_statistic_2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                               galcen.z.to_value(u.kpc)[mask],
                               e[mask] * eivar[mask], 
                               statistic='sum',
                               bins=zvz_bins)
    stat2 = binned_statistic_2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                               galcen.z.to_value(u.kpc)[mask],
                               eivar[mask], 
                               statistic='sum',
                               bins=zvz_bins)
    counts, *_ = np.histogram2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                                galcen.z.to_value(u.kpc)[mask],
                                bins=zvz_bins)
    H = stat1.statistic / stat2.statistic
    H[counts < 4] = np.nan
    vmin, vmax = np.nanpercentile(e[mask], [16, 84])

    cs = ax.pcolormesh(stat1.x_edge, stat1.y_edge, H.T, 
                       cmap='hesperia', vmin=vmin, vmax=vmax,
                       rasterized=True)
    
ax.set_xlim(-100, 100)
ax.set_ylim(-2, 2)

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

axes[0].set_title('full sample')
axes[1].set_title('main sequence')
axes[2].set_title('red giants')

cb = fig.colorbar(cs, ax=axes)

fig.set_facecolor('w')

---

# GALAH

In [ ]:
galah_rgb = galah.filter({'logg': (1, 3.5),
                          'teff': (3500, 5500)})

galah_ms = galah.filter({'logg': (3.5, 5),
                         'teff': (3500, 7500)})

In [ ]:
galah_mh_alpham_nodes = np.array([
    [0.6, -0.05],
    [0.6, 0.04],
    [0.15, 0.04],
    [-0.5, 0.13],
    [-0.9, 0.13],
    [-1., 0.07],
    [-0.2, -0.15],
    [0.2, -0.15],
    [0.6, -0.07]]
)
galah_mh_alpham_nodes[:, 1] += 0.04

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for i, d in enumerate([galah, galah_ms, galah_rgb]):
    ax = axes[i]

    H, xe, ye = np.histogram2d(d.t['teff'].value, 
                               d.t['logg'].value,
                               bins=(np.arange(3000, 8000+1e-3, 50),
                                     np.arange(0, 5.5+1e-3, 0.02)))

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)
    
ax.set_xlim(7500, 3000)
ax.set_ylim(5.5, 0)

#     ax.set_xlabel('metallicity, [M/H]')
#     ax.set_ylabel(r'alpha abundance, [$\alpha$/M]')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
for d in [galah_ms, galah_rgb]:
    fig, ax = plt.subplots(figsize=(8, 5))

    H, xe, ye = np.histogram2d(d.t['FE_H'], 
                               d.t['ALPHA_FE'],
                               bins=mh_am_bins)

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)

    ax.plot(galah_mh_alpham_nodes[:, 0], 
            galah_mh_alpham_nodes[:, 1],
            marker='', color='#777777', ls='--', lw=1.5)
    ax.set_xlim(-1.5, 0.7)
    ax.set_ylim(-0.12, 0.32)
    ax.set_xlabel('metallicity, [M/H]')
    ax.set_ylabel(r'alpha abundance, [$\alpha$/M]')

    fig.tight_layout()
    fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5.5), 
                         sharex=True, sharey=True)

for i, d in enumerate([galah, galah_ms, galah_rgb]):
    ax = axes[i]
    
    galcen = d.c.transform_to(galcen_frame)
    H, xe, ye = np.histogram2d(galcen.v_z.to_value(u.km/u.s), 
                               galcen.z.to_value(u.kpc),
                               bins=zvz_bins)

    ax.pcolormesh(xe, ye, H.T, norm=mpl.colors.LogNorm(), 
                  cmap=hesperia_r, rasterized=True)
    
ax.set_xlim(-100, 100)
ax.set_ylim(-2, 2)

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

axes[0].set_title('full sample')
axes[1].set_title('main sequence')
axes[2].set_title('red giants')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15.5, 5.5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

for i, d in enumerate([galah, galah_ms, galah_rgb]):
    ax = axes[i]
    
    e = d.get_elem_ratio('MG_FE')
    eivar = 1 / d.t['MG_FE_ERR']**2
    mask = (e > -3) & (e < 3)
    
    galcen = d.c.transform_to(galcen_frame)
    stat1 = binned_statistic_2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                               galcen.z.to_value(u.kpc)[mask],
                               e[mask] * eivar[mask], 
                               statistic='sum',
                               bins=zvz_bins)
    stat2 = binned_statistic_2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                               galcen.z.to_value(u.kpc)[mask],
                               eivar[mask], 
                               statistic='sum',
                               bins=zvz_bins)
    counts, *_ = np.histogram2d(galcen.v_z.to_value(u.km/u.s)[mask], 
                                galcen.z.to_value(u.kpc)[mask],
                                bins=zvz_bins)
    H = stat1.statistic / stat2.statistic
    H[counts < 4] = np.nan    
    vmin, vmax = np.nanpercentile(e[mask], [16, 84])

    cs = ax.pcolormesh(stat1.x_edge, stat1.y_edge, H.T, 
                       cmap='hesperia', vmin=vmin, vmax=vmax,
                       rasterized=True)
    
ax.set_xlim(-100, 100)
ax.set_ylim(-2, 2)

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')

axes[0].set_title('full sample')
axes[1].set_title('main sequence')
axes[2].set_title('red giants')

cb = fig.colorbar(cs, ax=axes)

fig.set_facecolor('w')